## Parameters

In [1]:
xgb_params = {
    'objective': 'multi:softmax', 
    'num_class': 10, 
    'n_estimators': 30, 
    'max_depth': 5, 
    'eta': 0.8
}

treelut_params = {
    'w_feature': 4, 
    'w_tree': 3, 
    'pipeline': [0, 1, 1], 
    'dir_path': './OutputFiles/MNIST/'}

## MNIST Data Loading

In [2]:
import numpy as np
from torchvision import datasets

data_train = datasets.MNIST(root="./Data", train=True, download=True)
data_test = datasets.MNIST(root="./Data", train=False, download=True)

X_train = np.array(data_train.data.numpy()).reshape(data_train.data.shape[0], -1)
y_train = np.array(data_train.targets.numpy())

X_test = np.array(data_test.data.numpy()).reshape(data_test.data.shape[0], -1)
y_test = np.array(data_test.targets.numpy())

## Data Quantization

In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
w_feature = treelut_params['w_feature']
X_train_quantized = np.round(scaler.fit_transform(X_train)*(2**w_feature-1))
X_test_quantized = np.clip(np.round(scaler.transform(X_test)*(2**w_feature-1)), 0, 2**w_feature-1)

## XGBoost Model Training

In [4]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xgb_clf = XGBClassifier(**xgb_params)
xgb_clf.fit(X_train_quantized, y_train)
y_pred_xgb = xgb_clf.predict(X_test_quantized)
print(f"XGB Model Accuracy: {accuracy_score(y_pred_xgb, y_test):.3f}")

XGB Model Accuracy: 0.971


## TreeLUT Model Generation

In [5]:
from treelut import TreeLUTClassifier

treelut_clf = TreeLUTClassifier(xgb_model=xgb_clf, **treelut_params)
treelut_clf.convert()
y_pred_treelut = treelut_clf.predict(X_test_quantized)
print(f"TreeLUT Model Accuracy: {accuracy_score(y_pred_treelut, y_test):.3f}")

treelut_clf.verilog()
treelut_clf.testbench(X_test_quantized, y_test)

TreeLUT Model Accuracy: 0.966
